<a href="https://colab.research.google.com/github/nunocesarsa/GoogleColab_public/blob/master/DSRP_Lunch_KFolding_ML_Simulated_inversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#loading up my google drive
from google.colab import drive
drive.mount('/content/drive')
#Perhaps this step can be skipped by saving directly to the workspace


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Loading up all the packages

In [0]:
#package instalation

#Installing PROSAIL
!pip install prosail

#latin hypercube stuff
#lets try to do a LHS
!pip install lhsmdu

#this package as a number of functions to deal with hyperspectral data
!pip install pysptools


In [0]:
#General purpose: 
import matplotlib.pyplot as plt
import numpy
import numpy as np
from mpl_toolkits.mplot3d import Axes3D

import pandas as pd

#PROSPECT+SAIL Radiative transfer mode package
import prosail

#Sampling design package
import lhsmdu

#package to for operations on spectral data
import pysptools as sptool 
from pysptools import distance


from sklearn.model_selection import KFold # import KFold

#machine learning stuff
#NEURAL NETWORK - Keras will be updated soon so this colab will also have to be changed

from sklearn.neural_network import MLPRegressor as ANN_reg #this is a simpler neural network package
from keras.models import Sequential, Model
from keras.layers import Dense, Input
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
#ignore the warning for now

#Random FOREST
# Fitting Random Forest Regression to the dataset 
# import the regressor 
from sklearn.ensemble import RandomForestRegressor 

#Gaussian processes
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import ConstantKernel, Matern, WhiteKernel, ExpSineSquared, RationalQuadratic, RBF

#aux functions
from sklearn.preprocessing import MinMaxScaler #this is to standardize the input data [not used for now]
from sklearn import metrics


# Generating a trait space

In [0]:
#number of samples
train_n3000 = 3000
train_n1500 = 1500
train_n0500 = 500

n_traits=4 #I will test on 4 varying traits: cab, car, cw,cm,lai

#generating a LHS hypercube (it uses a 0 to 1 interval that can be used as a multiplier against the different traits)
np.random.seed(0)
LHS_train3000 = lhsmdu.createRandomStandardUniformMatrix(n_traits,train_n3000 ) #the package has a more advanced method but it is too slow to process
LHS_train1500 = lhsmdu.createRandomStandardUniformMatrix(n_traits,train_n1500 )
LHS_train0500 = lhsmdu.createRandomStandardUniformMatrix(n_traits,train_n0500 )

#max_n=1 #this value should go from 1 to 2, so i make it change from 0 to 1 here and then add 1 later
max_cab=79. #add 1
#max_car=44. #add 1
#max_cbrown= 9.99 #add 0.01
max_cw=0.008 #add 0.001 
max_cm=0.008 #0.001
max_lai = 9.9 #add 0.1

min_cab = 10.
min_cw = 0.005
min_cm = 0.005
min_lai = .5



In [0]:
#preparing function inputs
pd_traits0500 = pd.DataFrame.transpose(pd.DataFrame(LHS_train0500))
pd_traits1500 = pd.DataFrame.transpose(pd.DataFrame(LHS_train1500))
pd_traits3000 = pd.DataFrame.transpose(pd.DataFrame(LHS_train3000))

pd_traits0500.columns = ["cab","cw","cm","lai"]
pd_traits1500.columns = ["cab","cw","cm","lai"]
pd_traits3000.columns = ["cab","cw","cm","lai"]

#pd_traits0500["car"]=pd_traits0500["car"]*max_car+1 
pd_traits0500["cab"]=pd_traits0500["cab"]*max_cab+min_cab
pd_traits0500["cw"] =pd_traits0500["cw"] *max_cw+min_cw
pd_traits0500["cm"] =pd_traits0500["cm"] *max_cm+min_cw
pd_traits0500["lai"]=pd_traits0500["lai"]*max_lai+min_lai

pd_traits1500["cab"]=pd_traits1500["cab"]*max_cab+min_cab
pd_traits1500["cw"] =pd_traits1500["cw"] *max_cw+min_cw
pd_traits1500["cm"] =pd_traits1500["cm"] *max_cm+min_cm
pd_traits1500["lai"]=pd_traits1500["lai"]*max_lai+min_lai

pd_traits3000["cab"]=pd_traits3000["cab"]*max_cab+min_cab
pd_traits3000["cw"] =pd_traits3000["cw"] *max_cw+min_cw
pd_traits3000["cm"] =pd_traits3000["cm"] *max_cm+min_cm
pd_traits3000["lai"]=pd_traits3000["lai"]*max_lai+min_lai

#lets also create a numpy object for the tratis
np_traits0500 = pd_traits0500.iloc[:,:].values
np_traits1500 = pd_traits1500.iloc[:,:].values
np_traits3000 = pd_traits3000.iloc[:,:].values


Now that we have the trait space, we need to generate the spectral space. For that we need 3 custom functions:



1.   to call prosail given a pre-set group of values of trait
2.   to convert hyperspectral to S2 resolution using spectral convolution
3.   to iterate through the entire trait space while generating the spectra


In [0]:
def custom_prosail(cab,cw,cm,lai):
  import prosail
  #default parameters
  n= 1.
  car=10.
  cbrown=0.01
  typelidf=1 #this is the default option
  lidfa = -1 #leaf angle distribution parameter a and b
  lidfb=-0.15
  hspot= 0.01 #hotspot parameters - got this from R package https://www.rdocumentation.org/packages/hsdar/versions/0.4.1/topics/PROSAIL
  #sun and viewing angle
  tts=30. #observation and solar position parameters
  tto=10. 
  psi=0.
  #for now i put them by hand but they should be an input of a custom function
  #tts=sol_zen #solar zenith angle
  #tto=inc_zen #sensor zenith angle
  #psi=raa
  rho_out = prosail.run_prosail(n,
                                 cab,
                                 car,
                                 cbrown,
                                 cw,
                                 cm,
                                 lai,
                                 lidfa,
                                 hspot,
                                 tts,tto,psi,
                                 typelidf, lidfb,
                                 prospect_version="D",
                                 factor='SDR', 
                                 rsoil=.5, psoil=.5)
  return(rho_out)

In [0]:
def Prosail2S2(path2csv,spectra_input):
  #importing pandas
  import pandas as pd
  import numpy
  import numpy as np
  #upload a S2_Response.csv from https://earth.esa.int/web/sentinel/user-guides/sentinel-2-msi/document-library/-/asset_publisher/Wk0TKajiISaR/content/sentinel-2a-spectral-responses

  s2_table = pd.read_csv(path2csv,sep=";",decimal=",") #check if this is proper, regarding the sep and dec
  #chekc which row you are actually extracting

  s2_table_sel = s2_table[s2_table['SR_WL'].between(400,2500)] #selects all values between 400 and 2500
  spectra_input_df = pd.DataFrame(data=spectra_input,columns=["rho"],index=s2_table_sel.index) #transforms the input array into a pandas df with the column name rho and row.index = to the original input table

  
  rho_s2 = s2_table_sel.multiply(spectra_input_df['rho'],axis="index") #calculates the numerator
  w_band_sum = s2_table_sel.sum(axis=0,skipna = True) #calculates the denominator

  output = (rho_s2.sum(axis=0)/w_band_sum).rename_axis("ID").values #runs the weighted mean and converts the output to a numpy array

  return output[1:] #removes the first value because it represents the wavelength column

#please LOAD THTE FILE NOW
filepath="/content/drive/My Drive/S2_Response.csv"
#filepath="/content/S2_Responses_S2B.csv"
#filepath="/content/drive/My Drive/S2_Response.csv"



In [0]:
def Gen_spectra_data(traits):
  k = 1
  #pd_train_traits=traits
  #print(range(len(traits)))
  for i in range(len(traits)):
    #n_t = pd_train_traits["n"][i]
    cab_t = traits["cab"][i]
    #car_t = pd_train_traits["car"][i]
    #cbrown_t = pd_train_traits["cbrown"][i]
    cw_t = traits["cw"][i]
    cm_t = traits["cm"][i]
    lai_t = traits["lai"][i]

    if k == 1:
      tr_rho_s = custom_prosail(cab_t,cw_t,cm_t,lai_t)
      tr_rho_s = Prosail2S2(filepath,tr_rho_s)
      #plt.plot ( x, tr_rho_s, ':', label="Training prosail")
      #plt.legend(loc='best')
      
    if k > 1:
      tr_rho_t = custom_prosail(cab_t,cw_t,cm_t,lai_t)
      tr_rho_t = Prosail2S2(filepath,tr_rho_t)
      tr_rho_s = np.vstack((tr_rho_s,tr_rho_t))
      #plt.plot ( x, tr_rho_t, ':')

    k = k+1


  rho_samples=tr_rho_s


  return rho_samples

This generates the 3 different sample sets 

In [0]:
#pd_train_traits["n"]=pd_t
np_spectra0500 = Gen_spectra_data(pd_traits0500)
np_spectra1500 = Gen_spectra_data(pd_traits1500)
np_spectra3000 = Gen_spectra_data(pd_traits3000)

print(np_spectra0500.shape)
print(np_spectra1500.shape)
print(np_spectra3000.shape)

#selects only the equivalent 20m bands
train_df_0500 = np_spectra0500[:,[1,2,3,4,5,6,8,11,12]]
train_df_1500 = np_spectra1500[:,[1,2,3,4,5,6,8,11,12]]
train_df_3000 = np_spectra3000[:,[1,2,3,4,5,6,8,11,12]]

(500, 13)
(1500, 13)
(3000, 13)


# Setting up the models 

In [0]:
#Random forest
#initializing the random forest
rfr_ml = RandomForestRegressor(n_estimators=1000,random_state=0,
                              min_samples_leaf=5,min_samples_split=5,verbose=1)

#single task Artificial neural network
ann_ml = Sequential()
#ann_ml.add(Dense(9, input_dim=9, activation='linear'))
ann_ml.add(Dense(10, input_dim=9, activation='tanh'))
ann_ml.add(Dense(6, activation='relu'))
ann_ml.add(Dense(4)) #indeed this ha to be added in this case without any activ function, the R script added this on its own
#model.add(Dense(1, activation='sigmoid'))

# compile the keras model
ann_ml.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

#gaussian processes with default acquisition function
#initializing the gaussian process
gpr_ml = GaussianProcessRegressor(n_restarts_optimizer=50,
                                        normalize_y=True,
                                        random_state=0)

#multi-task neural network - 32 by 16 by 8 per task
inputs = Input(shape=(9,))
sub1 = Dense(32, activation='tanh')(inputs)
sub2 = Dense(16, activation='tanh')(sub1)
cab1 = Dense(8, activation='sigmoid')(sub2)
cw1  = Dense(8, activation='sigmoid')(sub2)
cm1  = Dense(8, activation='sigmoid')(sub2)
lai1 = Dense(8, activation='sigmoid')(sub2)
cab2 = Dense(1, activation='linear')(cab1)
cw2  = Dense(1, activation='linear')(cw1)
cm2  = Dense(1, activation='linear')(cm1)
lai2 = Dense(1, activation='linear')(lai1)


#building the model using keras api
model = Model(inputs=inputs, outputs=[cab2,cw2,cm2,lai2])
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])



Before going into the loop, we set up a place for storing the output statistics

In [0]:
column_names=["Model",
              "NSamples",
              "Variable",
              "Fold_nr",
              "ExplVar",
              "Max_err",
              "Mean_abs_Err",
              "Mean_sqr_err",
              #"Mean_sqr_lg_err",
              "Median_abs_err",
              "r2",
              "MAPE"]
              #"Mean_poiss_dev",
              #"Mean_gamma_dev"]
              #"Mean_tweed_dev"]

#mape is not existant in the package so we have to create it:
#https://stats.stackexchange.com/questions/58391/mean-absolute-percentage-error-mape-in-scikit-learn
#from sklearn.utils import check_array
def mean_absolute_percentage_error(y_true, y_pred): 

    ## Note: does not handle mix 1d representation
    #if _is_1d(y_true): 
    #    y_true, y_pred = _check_1d_array(y_true, y_pred)

    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

  #Here, the file is used for saving
  #creating a df to receive the data
df_metrics = pd.DataFrame(columns=column_names)

#this command is enough to set u the k-fold
kf = KFold(n_splits=5,shuffle=True,random_state=0) # Define the split 

Running for 500 training samples

In [0]:

#this is a function to re-scale inputs for the neural networks
scaler_0500 = MinMaxScaler()

#with 500 samples
k=1
for train_index, test_index in kf.split(train_df_0500):

  #subsetting for ith k-fold
  X_train, X_test = train_df_0500[train_index], train_df_0500[test_index]
  Y_train, Y_test = np_traits0500[train_index], np_traits0500[test_index]
  label_names = ["cab","cw","cm","lai"]

  #RF - Training 
  rfr_ml.fit(X_train,Y_train)

  #ANN - Training 
  scaler_0500.fit(Y_train)
  Y_train_norm = scaler_0500.transform(Y_train)
  ann_ml.fit(X_train,Y_train_norm,epochs=500,verbose=0)

  #GPR - Training 
  gpr_ml.fit(X_train,Y_train)

  #Multi task ANN training
  cab_train = Y_train_norm[:,0]
  cw_train = Y_train_norm[:,1]
  cm_train = Y_train_norm[:,2]
  lai_train = Y_train_norm[:,3]
  
  model.fit(X_train,[cab_train,cw_train,cm_train,lai_train],epochs=100,verbose=0)#hides spam on the output

  #Prediction
  y_ann_0500 = scaler_0500.inverse_transform(ann_ml.predict(X_test))
  y_rfr_0500 = rfr_ml.predict(X_test)
  y_gpr_0500 = gpr_ml.predict(X_test)

  #the multi task network requires a bit more effort
  Y_mtn_0500 = model.predict(X_test)


  #then we create a stack
  cab_pred = Y_mtn_0500[0]
  cw_pred  = Y_mtn_0500[1]
  cm_pred  = Y_mtn_0500[2]
  lai_pred = Y_mtn_0500[3]
  np_Y_ann_pred = np.array(cab_pred)
  np_Y_ann_pred = np.hstack((np_Y_ann_pred,cw_pred))
  np_Y_ann_pred = np.hstack((np_Y_ann_pred,cm_pred))
  np_Y_ann_pred = np.hstack((np_Y_ann_pred,lai_pred))

  #then we invert the transformation and store to a numpy
  y_mtn_0500 = scaler_0500.inverse_transform(np_Y_ann_pred)


  for i in range(n_traits):

    #creating the temp list
    ann_temp_list = {"Model":"ANN",
                     "NSamples":500,
                     "Variable":label_names[i],
                     "Fold_nr":k,
                     "ExplVar": metrics.explained_variance_score(Y_test[:,i], y_ann_0500[:,i]),
                     "Max_err": metrics.max_error(Y_test[:,i], y_ann_0500[:,i]),
                     "Mean_abs_Err": metrics.mean_absolute_error(Y_test[:,i], y_ann_0500[:,i]),
                     "Mean_sqr_err": metrics.mean_squared_error(Y_test[:,i], y_ann_0500[:,i]),
                     #"Mean_sqr_lg_err": metrics.mean_squared_log_error(Y_test[:,i], y_ann_0500[:,i]),
                     "Median_abs_err" : metrics.median_absolute_error(Y_test[:,i], y_ann_0500[:,i]),
                     "r2": metrics.r2_score(Y_test[:,i], y_ann_0500[:,i]),
                     #"Mean_poiss_dev" : metrics.mean_poisson_deviance(Y_test[:,i], y_ann_0500[:,i]),
                     #"Mean_gamma_dev" : metrics.mean_gamma_deviance(Y_test[:,i], y_ann_0500[:,i])}
                     #"Mean_tweed_dev" : metrics.mean_tweedie_deviance(Y_test[:,i], y_ann_0500[:,i])}
                     "MAPE": mean_absolute_percentage_error(Y_test[:,i], y_ann_0500[:,i])}

    rfr_temp_list = {"Model":"RFr",
                     "NSamples":500,
                     "Variable":label_names[i],
                     "Fold_nr":k,
                     "ExplVar": metrics.explained_variance_score(Y_test[:,i], y_rfr_0500[:,i]),
                     "Max_err": metrics.max_error(Y_test[:,i], y_rfr_0500[:,i]),
                     "Mean_abs_Err": metrics.mean_absolute_error(Y_test[:,i], y_rfr_0500[:,i]),
                     "Mean_sqr_err": metrics.mean_squared_error(Y_test[:,i], y_rfr_0500[:,i]),
                     #"Mean_sqr_lg_err": metrics.mean_squared_log_error(Y_test[:,i], y_rfr_0500[:,i]),
                     "Median_abs_err" : metrics.median_absolute_error(Y_test[:,i], y_rfr_0500[:,i]),
                     "r2": metrics.r2_score(Y_test[:,i], y_rfr_0500[:,i]),
                     #"Mean_poiss_dev" : metrics.mean_poisson_deviance(Y_test[:,i], y_rfr_0500[:,i]),
                     #"Mean_gamma_dev" : metrics.mean_gamma_deviance(Y_test[:,i], y_rfr_0500[:,i])}
                     #"Mean_tweed_dev" : metrics.mean_tweedie_deviance(Y_test[:,i], y_rfr_0500[:,i])}
                     "MAPE": mean_absolute_percentage_error(Y_test[:,i], y_rfr_0500[:,i])}

    gpr_temp_list = {"Model":"GPR",
                     "NSamples":500,
                     "Variable":label_names[i],
                     "Fold_nr":k,
                     "ExplVar": metrics.explained_variance_score(Y_test[:,i], y_gpr_0500[:,i]),
                     "Max_err": metrics.max_error(Y_test[:,i], y_gpr_0500[:,i]),
                     "Mean_abs_Err": metrics.mean_absolute_error(Y_test[:,i], y_gpr_0500[:,i]),
                     "Mean_sqr_err": metrics.mean_squared_error(Y_test[:,i], y_gpr_0500[:,i]),
                     #"Mean_sqr_lg_err": metrics.mean_squared_log_error(Y_test[:,i], y_gpr_0500[:,i]),
                     "Median_abs_err" : metrics.median_absolute_error(Y_test[:,i], y_gpr_0500[:,i]),
                     "r2": metrics.r2_score(Y_test[:,i], y_gpr_0500[:,i]),
                     #"Mean_poiss_dev" : metrics.mean_poisson_deviance(Y_test[:,i], y_gpr_0500[:,i]),
                     #"Mean_gamma_dev" : metrics.mean_gamma_deviance(Y_test[:,i], y_gpr_0500[:,i])}
                     #"Mean_tweed_dev" : metrics.mean_tweedie_deviance(Y_test[:,i], y_gpr_0500[:,i])}
                     "MAPE": mean_absolute_percentage_error(Y_test[:,i], y_gpr_0500[:,i])}

    mtn_temp_list = {"Model":"mtn",
                     "NSamples":500,
                     "Variable":label_names[i],
                     "Fold_nr":k,
                     "ExplVar": metrics.explained_variance_score(Y_test[:,i], y_mtn_0500[:,i]),
                     "Max_err": metrics.max_error(Y_test[:,i], y_mtn_0500[:,i]),
                     "Mean_abs_Err": metrics.mean_absolute_error(Y_test[:,i], y_mtn_0500[:,i]),
                     "Mean_sqr_err": metrics.mean_squared_error(Y_test[:,i], y_mtn_0500[:,i]),
                     #"Mean_sqr_lg_err": metrics.mean_squared_log_error(Y_test[:,i], y_gpr_0500[:,i]),
                     "Median_abs_err" : metrics.median_absolute_error(Y_test[:,i], y_mtn_0500[:,i]),
                     "r2": metrics.r2_score(Y_test[:,i], y_mtn_0500[:,i]),
                     #"Mean_poiss_dev" : metrics.mean_poisson_deviance(Y_test[:,i], y_gpr_0500[:,i]),
                     #"Mean_gamma_dev" : metrics.mean_gamma_deviance(Y_test[:,i], y_gpr_0500[:,i])}
                     #"Mean_tweed_dev" : metrics.mean_tweedie_deviance(Y_test[:,i], y_gpr_0500[:,i])}
                     "MAPE": mean_absolute_percentage_error(Y_test[:,i], y_mtn_0500[:,i])}

    #appending to the dataframe
    df_metrics = df_metrics.append(ann_temp_list,ignore_index=True)
    df_metrics = df_metrics.append(rfr_temp_list,ignore_index=True)
    df_metrics = df_metrics.append(gpr_temp_list,ignore_index=True)
    df_metrics = df_metrics.append(mtn_temp_list,ignore_index=True)

  k = k+1

df_metrics.to_csv("/content/drive/My Drive/DSRP_Lunch_outputs/KFold_0500_4ML.csv",sep=";",decimal=",")

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    1.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    1.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    1.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[

In [0]:
#df_metrics

For 1500 samples

In [0]:
#the ANN requires that we transform the variables
from sklearn.preprocessing import MinMaxScaler 
scaler_1500 = MinMaxScaler()

#with 1500 samples
k=1
for train_index, test_index in kf.split(train_df_1500):

  print("Now in fold nr:", k)

  #subsetting for ith k-fold
  X_train, X_test = train_df_1500[train_index], train_df_1500[test_index]
  Y_train, Y_test = np_traits1500[train_index], np_traits1500[test_index]
  label_names = ["cab","cw","cm","lai"]

  #ANN - Training 
  scaler_1500.fit(Y_train)
  Y_train_norm = scaler_1500.transform(Y_train)
  ann_ml.fit(X_train,Y_train_norm,epochs=1500,verbose=0)

  #RF - Training n
  rfr_ml.fit(X_train,Y_train)

  #GPR - Training 
  gpr_ml.fit(X_train,Y_train)

  #Multi task ANN training
  cab_train = Y_train_norm[:,0]
  cw_train = Y_train_norm[:,1]
  cm_train = Y_train_norm[:,2]
  lai_train = Y_train_norm[:,3]
  
  model.fit(X_train,[cab_train,cw_train,cm_train,lai_train],epochs=100,verbose=0)#hides spam on the output

  #Prediction
  y_ann_1500 = scaler_1500.inverse_transform(ann_ml.predict(X_test))
  y_rfr_1500 = rfr_ml.predict(X_test)
  y_gpr_1500 = gpr_ml.predict(X_test)

  #the multi task network requires a bit more effort
  Y_mtn_1500 = model.predict(X_test)

  #then we create a stack
  cab_pred = Y_mtn_1500[0]
  cw_pred  = Y_mtn_1500[1]
  cm_pred  = Y_mtn_1500[2]
  lai_pred = Y_mtn_1500[3]
  np_Y_ann_pred = np.array(cab_pred)
  np_Y_ann_pred = np.hstack((np_Y_ann_pred,cw_pred))
  np_Y_ann_pred = np.hstack((np_Y_ann_pred,cm_pred))
  np_Y_ann_pred = np.hstack((np_Y_ann_pred,lai_pred))

  #then we invert the transformation and store to a numpy
  y_mtn_1500 = scaler_1500.inverse_transform(np_Y_ann_pred)


  for i in range(n_traits):

    #creating the temp list
    ann_temp_list = {"Model":"ANN",
                     "NSamples":1500,
                     "Variable":label_names[i],
                     "Fold_nr":k,
                     "ExplVar": metrics.explained_variance_score(Y_test[:,i], y_ann_1500[:,i]),
                     "Max_err": metrics.max_error(Y_test[:,i], y_ann_1500[:,i]),
                     "Mean_abs_Err": metrics.mean_absolute_error(Y_test[:,i], y_ann_1500[:,i]),
                     "Mean_sqr_err": metrics.mean_squared_error(Y_test[:,i], y_ann_1500[:,i]),
                     #"Mean_sqr_lg_err": metrics.mean_squared_log_error(Y_test[:,i], y_ann_1500[:,i]),
                     "Median_abs_err" : metrics.median_absolute_error(Y_test[:,i], y_ann_1500[:,i]),
                     "r2": metrics.r2_score(Y_test[:,i], y_ann_1500[:,i]),
                     #"Mean_poiss_dev" : metrics.mean_poisson_deviance(Y_test[:,i], y_ann_1500[:,i]),
                     #"Mean_gamma_dev" : metrics.mean_gamma_deviance(Y_test[:,i], y_ann_1500[:,i])}
                     #"Mean_tweed_dev" : metrics.mean_tweedie_deviance(Y_test[:,i], y_ann_1500[:,i])}
                     "MAPE": mean_absolute_percentage_error(Y_test[:,i], y_ann_1500[:,i])}

    rfr_temp_list = {"Model":"RFr",
                     "NSamples":1500,
                     "Variable":label_names[i],
                     "Fold_nr":k,
                     "ExplVar": metrics.explained_variance_score(Y_test[:,i], y_rfr_1500[:,i]),
                     "Max_err": metrics.max_error(Y_test[:,i], y_rfr_1500[:,i]),
                     "Mean_abs_Err": metrics.mean_absolute_error(Y_test[:,i], y_rfr_1500[:,i]),
                     "Mean_sqr_err": metrics.mean_squared_error(Y_test[:,i], y_rfr_1500[:,i]),
                     #"Mean_sqr_lg_err": metrics.mean_squared_log_error(Y_test[:,i], y_rfr_1500[:,i]),
                     "Median_abs_err" : metrics.median_absolute_error(Y_test[:,i], y_rfr_1500[:,i]),
                     "r2": metrics.r2_score(Y_test[:,i], y_rfr_1500[:,i]),
                     #"Mean_poiss_dev" : metrics.mean_poisson_deviance(Y_test[:,i], y_rfr_1500[:,i]),
                     #"Mean_gamma_dev" : metrics.mean_gamma_deviance(Y_test[:,i], y_rfr_1500[:,i])}
                     #"Mean_tweed_dev" : metrics.mean_tweedie_deviance(Y_test[:,i], y_rfr_1500[:,i])}
                     "MAPE": mean_absolute_percentage_error(Y_test[:,i], y_rfr_1500[:,i])}

    gpr_temp_list = {"Model":"GPR",
                     "NSamples":1500,
                     "Variable":label_names[i],
                     "Fold_nr":k,
                     "ExplVar": metrics.explained_variance_score(Y_test[:,i], y_gpr_1500[:,i]),
                     "Max_err": metrics.max_error(Y_test[:,i], y_gpr_1500[:,i]),
                     "Mean_abs_Err": metrics.mean_absolute_error(Y_test[:,i], y_gpr_1500[:,i]),
                     "Mean_sqr_err": metrics.mean_squared_error(Y_test[:,i], y_gpr_1500[:,i]),
                     #"Mean_sqr_lg_err": metrics.mean_squared_log_error(Y_test[:,i], y_gpr_1500[:,i]),
                     "Median_abs_err" : metrics.median_absolute_error(Y_test[:,i], y_gpr_1500[:,i]),
                     "r2": metrics.r2_score(Y_test[:,i], y_gpr_1500[:,i]),
                     #"Mean_poiss_dev" : metrics.mean_poisson_deviance(Y_test[:,i], y_gpr_1500[:,i]),
                     #"Mean_gamma_dev" : metrics.mean_gamma_deviance(Y_test[:,i], y_gpr_1500[:,i])}
                     #"Mean_tweed_dev" : metrics.mean_tweedie_deviance(Y_test[:,i], y_gpr_1500[:,i])}
                     "MAPE": mean_absolute_percentage_error(Y_test[:,i], y_gpr_1500[:,i])}

    mtn_temp_list = {"Model":"mtn",
                     "NSamples":1500,
                     "Variable":label_names[i],
                     "Fold_nr":k,
                     "ExplVar": metrics.explained_variance_score(Y_test[:,i], y_mtn_1500[:,i]),
                     "Max_err": metrics.max_error(Y_test[:,i], y_mtn_1500[:,i]),
                     "Mean_abs_Err": metrics.mean_absolute_error(Y_test[:,i], y_mtn_1500[:,i]),
                     "Mean_sqr_err": metrics.mean_squared_error(Y_test[:,i], y_mtn_1500[:,i]),
                     #"Mean_sqr_lg_err": metrics.mean_squared_log_error(Y_test[:,i], y_gpr_0500[:,i]),
                     "Median_abs_err" : metrics.median_absolute_error(Y_test[:,i], y_mtn_1500[:,i]),
                     "r2": metrics.r2_score(Y_test[:,i], y_mtn_1500[:,i]),
                     #"Mean_poiss_dev" : metrics.mean_poisson_deviance(Y_test[:,i], y_gpr_0500[:,i]),
                     #"Mean_gamma_dev" : metrics.mean_gamma_deviance(Y_test[:,i], y_gpr_0500[:,i])}
                     #"Mean_tweed_dev" : metrics.mean_tweedie_deviance(Y_test[:,i], y_gpr_0500[:,i])}
                     "MAPE": mean_absolute_percentage_error(Y_test[:,i], y_mtn_1500[:,i])}

  #appending to the dataframe
    df_metrics = df_metrics.append(ann_temp_list,ignore_index=True)
    df_metrics = df_metrics.append(rfr_temp_list,ignore_index=True)
    df_metrics = df_metrics.append(gpr_temp_list,ignore_index=True)
    df_metrics = df_metrics.append(mtn_temp_list,ignore_index=True)


  k = k+1

df_metrics.to_csv("/content/drive/My Drive/DSRP_Lunch_outputs/KFold_1500_4ML.csv",sep=";",decimal=",")


Now in fold nr: 1


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    5.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.1s finished


Now in fold nr: 2


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    5.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.1s finished


Now in fold nr: 3


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    5.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.1s finished


Now in fold nr: 4


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    5.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.1s finished


Now in fold nr: 5


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    5.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.1s finished


In [0]:
#df_metrics

,Model,NSamples,Variable,Fold_nr,ExplVar,Max_err,Mean_abs_Err,Mean_sqr_err,Median_abs_err,r2,MAPE
0,ANN,500,cab,1,0.991503,8.288998,1.677632,5.027419e+00,1.212710,0.991363,4.933098
1,RFr,500,cab,1,0.992678,14.155291,1.241351,4.262245e+00,0.892719,0.992678,3.137953
2,GPR,500,cab,1,0.999963,1.067922,0.059297,2.270711e-02,0.024714,0.999961,0.264224
3,mtn,500,cab,1,0.997711,4.326161,0.815881,1.385892e+00,0.519396,0.997619,2.194434
4,ANN,500,cw,1,0.970710,0.001578,0.000301,1.757736e-07,0.000225,0.969987,3.816482
...,...,...,...,...,...,...,...,...,...,...,...
155,mtn,1500,cm,5,0.994836,0.001495,0.000109,2.743395e-08,0.000077,0.994835,1.266226
156,ANN,1500,lai,5,0.991781,0.976543,0.207938,7.262601e-02,0.162331,0.991775,6.438858
157,RFr,1500,lai,5,0.916048,3.904220,0.608495,7.431306e-01,0.410513,0.915837,21.342844
158,GPR,1500,lai,5,0.999983,0.089208,0.008898,1.540369e-04,0.006586,0.999983,0.373101


For 3000 samples





In [0]:
scaler_3000 = MinMaxScaler()
#with 500 samples
k=1

for train_index, test_index in kf.split(train_df_3000):

  #subsetting for ith k-fold
  X_train, X_test = train_df_3000[train_index], train_df_3000[test_index]
  Y_train, Y_test = np_traits3000[train_index], np_traits3000[test_index]
  label_names = ["cab","cw","cm","lai"]

  #ANN - Training 
  scaler_3000.fit(Y_train)
  Y_train_norm = scaler_3000.transform(Y_train)
  ann_ml.fit(X_train,Y_train_norm,epochs=3000,verbose=0)

  #RF - Training n
  rfr_ml.fit(X_train,Y_train)

  #GPR - Training 
  gpr_ml.fit(X_train,Y_train)


  #Multi task ANN training
  cab_train = Y_train_norm[:,0]
  cw_train = Y_train_norm[:,1]
  cm_train = Y_train_norm[:,2]
  lai_train = Y_train_norm[:,3]
  
  model.fit(X_train,[cab_train,cw_train,cm_train,lai_train],epochs=100,verbose=0)#hides spam on the output

  #Prediction
  y_ann_3000 = scaler_0500.inverse_transform(ann_ml.predict(X_test))
  y_rfr_3000 = rfr_ml.predict(X_test)
  y_gpr_3000 = gpr_ml.predict(X_test)

  #the multi task network requires a bit more effort
  Y_mtn_3000 = model.predict(X_test)

  #then we create a stack
  cab_pred = Y_mtn_3000[0]
  cw_pred  = Y_mtn_3000[1]
  cm_pred  = Y_mtn_3000[2]
  lai_pred = Y_mtn_3000[3]
  np_Y_ann_pred = np.array(cab_pred)
  np_Y_ann_pred = np.hstack((np_Y_ann_pred,cw_pred))
  np_Y_ann_pred = np.hstack((np_Y_ann_pred,cm_pred))
  np_Y_ann_pred = np.hstack((np_Y_ann_pred,lai_pred))

  #then we invert the transformation and store to a numpy
  y_mtn_3000 = scaler_3000.inverse_transform(np_Y_ann_pred)



  for i in range(n_traits):

    #creating the temp list
    ann_temp_list = {"Model":"ANN",
                     "NSamples":3000,
                     "Variable":label_names[i],
                     "Fold_nr":k,
                     "ExplVar": metrics.explained_variance_score(Y_test[:,i], y_ann_3000[:,i]),
                     "Max_err": metrics.max_error(Y_test[:,i], y_ann_3000[:,i]),
                     "Mean_abs_Err": metrics.mean_absolute_error(Y_test[:,i], y_ann_3000[:,i]),
                     "Mean_sqr_err": metrics.mean_squared_error(Y_test[:,i], y_ann_3000[:,i]),
                     #"Mean_sqr_lg_err": metrics.mean_squared_log_error(Y_test[:,i], y_ann_3000[:,i]),
                     "Median_abs_err" : metrics.median_absolute_error(Y_test[:,i], y_ann_3000[:,i]),
                     "r2": metrics.r2_score(Y_test[:,i], y_ann_3000[:,i]),
                     #"Mean_poiss_dev" : metrics.mean_poisson_deviance(Y_test[:,i], y_ann_3000[:,i]),
                     #"Mean_gamma_dev" : metrics.mean_gamma_deviance(Y_test[:,i], y_ann_3000[:,i])}
                     #"Mean_tweed_dev" : metrics.mean_tweedie_deviance(Y_test[:,i], y_ann_3000[:,i])}
                     "MAPE": mean_absolute_percentage_error(Y_test[:,i], y_ann_3000[:,i])}

    rfr_temp_list = {"Model":"RFr",
                     "NSamples":3000,
                     "Variable":label_names[i],
                     "Fold_nr":k,
                     "ExplVar": metrics.explained_variance_score(Y_test[:,i], y_rfr_3000[:,i]),
                     "Max_err": metrics.max_error(Y_test[:,i], y_rfr_3000[:,i]),
                     "Mean_abs_Err": metrics.mean_absolute_error(Y_test[:,i], y_rfr_3000[:,i]),
                     "Mean_sqr_err": metrics.mean_squared_error(Y_test[:,i], y_rfr_3000[:,i]),
                     #"Mean_sqr_lg_err": metrics.mean_squared_log_error(Y_test[:,i], y_rfr_3000[:,i]),
                     "Median_abs_err" : metrics.median_absolute_error(Y_test[:,i], y_rfr_3000[:,i]),
                     "r2": metrics.r2_score(Y_test[:,i], y_rfr_3000[:,i]),
                     #"Mean_poiss_dev" : metrics.mean_poisson_deviance(Y_test[:,i], y_rfr_3000[:,i]),
                     #"Mean_gamma_dev" : metrics.mean_gamma_deviance(Y_test[:,i], y_rfr_3000[:,i])}
                     #"Mean_tweed_dev" : metrics.mean_tweedie_deviance(Y_test[:,i], y_rfr_3000[:,i])}
                     "MAPE": mean_absolute_percentage_error(Y_test[:,i], y_rfr_3000[:,i])}

    gpr_temp_list = {"Model":"GPR",
                     "NSamples":3000,
                     "Variable":label_names[i],
                     "Fold_nr":k,
                     "ExplVar": metrics.explained_variance_score(Y_test[:,i], y_gpr_3000[:,i]),
                     "Max_err": metrics.max_error(Y_test[:,i], y_gpr_3000[:,i]),
                     "Mean_abs_Err": metrics.mean_absolute_error(Y_test[:,i], y_gpr_3000[:,i]),
                     "Mean_sqr_err": metrics.mean_squared_error(Y_test[:,i], y_gpr_3000[:,i]),
                     #"Mean_sqr_lg_err": metrics.mean_squared_log_error(Y_test[:,i], y_gpr_3000[:,i]),
                     "Median_abs_err" : metrics.median_absolute_error(Y_test[:,i], y_gpr_3000[:,i]),
                     "r2": metrics.r2_score(Y_test[:,i], y_gpr_3000[:,i]),
                     #"Mean_poiss_dev" : metrics.mean_poisson_deviance(Y_test[:,i], y_gpr_3000[:,i]),
                     #"Mean_gamma_dev" : metrics.mean_gamma_deviance(Y_test[:,i], y_gpr_3000[:,i])}
                     #"Mean_tweed_dev" : metrics.mean_tweedie_deviance(Y_test[:,i], y_gpr_3000[:,i])}
                     "MAPE": mean_absolute_percentage_error(Y_test[:,i], y_gpr_3000[:,i])}

    mtn_temp_list = {"Model":"mtn",
                     "NSamples":3000,
                     "Variable":label_names[i],
                     "Fold_nr":k,
                     "ExplVar": metrics.explained_variance_score(Y_test[:,i], y_mtn_3000[:,i]),
                     "Max_err": metrics.max_error(Y_test[:,i], y_mtn_3000[:,i]),
                     "Mean_abs_Err": metrics.mean_absolute_error(Y_test[:,i], y_mtn_3000[:,i]),
                     "Mean_sqr_err": metrics.mean_squared_error(Y_test[:,i], y_mtn_3000[:,i]),
                     #"Mean_sqr_lg_err": metrics.mean_squared_log_error(Y_test[:,i], y_gpr_0500[:,i]),
                     "Median_abs_err" : metrics.median_absolute_error(Y_test[:,i], y_mtn_3000[:,i]),
                     "r2": metrics.r2_score(Y_test[:,i], y_mtn_3000[:,i]),
                     #"Mean_poiss_dev" : metrics.mean_poisson_deviance(Y_test[:,i], y_gpr_0500[:,i]),
                     #"Mean_gamma_dev" : metrics.mean_gamma_deviance(Y_test[:,i], y_gpr_0500[:,i])}
                     #"Mean_tweed_dev" : metrics.mean_tweedie_deviance(Y_test[:,i], y_gpr_0500[:,i])}
                     "MAPE": mean_absolute_percentage_error(Y_test[:,i], y_mtn_3000[:,i])}

  #appending to the dataframe
    df_metrics = df_metrics.append(ann_temp_list,ignore_index=True)
    df_metrics = df_metrics.append(rfr_temp_list,ignore_index=True)
    df_metrics = df_metrics.append(gpr_temp_list,ignore_index=True)
    df_metrics = df_metrics.append(mtn_temp_list,ignore_index=True)
  k = k+1

df_metrics.to_csv("/content/drive/My Drive/DSRP_Lunch_outputs/KFold_3000_4ML.csv",sep=";",decimal=",")

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:   11.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:   11.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:   11.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[

In [0]:
#df_metrics

The exploratory plots are made in R